In [1]:
import numpy as np
import math
import pandas as pd
from geopy import distance
from itertools import combinations
import folium
import ast
import time
#import timeit
# start_time=time.time()
# print(time.time()-start_time)

In [2]:
file_path='/home/dataplus-01/Documents/Modules_Socle/Python/Projet_arbre/data/trees.csv'
data = pd.read_csv(file_path, sep=",", header = 0, index_col=False,names=None)
# on peut également appliquer un converter au moment de read, ou encore de read qu'une colonne
# df = pd.read_csv('/home/dataplus-01/Documents/Modules_Socle/Python/Projet_arbre/data/trees copy.csv'
#                  , converters={'GeoJSON': pd.read_json})

In [3]:
# on identifie la colonne qui nous intéresse
data = pd.DataFrame(data = data)
GeoJSON=data["GeoJSON"]
GeoJSON

0        {"type":"Point","coordinates":[5.7603469008942...
1        {"type":"Point","coordinates":[5.7598264646441...
2        {"type":"Point","coordinates":[5.7599807314486...
3        {"type":"Point","coordinates":[5.7600570301267...
4        {"type":"Point","coordinates":[5.7600202792924...
                               ...                        
31238    {"type":"Point","coordinates":[5.7117169490564...
31239    {"type":"Point","coordinates":[5.7115904446110...
31240    {"type":"Point","coordinates":[5.7114873970721...
31241    {"type":"Point","coordinates":[5.7117859768817...
31242    {"type":"Point","coordinates":[5.7113357012675...
Name: GeoJSON, Length: 31243, dtype: object

In [4]:
# on la reformate comme il faut
GeoJSON_dict=GeoJSON.map(lambda a:ast.literal_eval(a))
type(GeoJSON_dict.iloc[0])

dict

In [5]:
# test d'accés à l'information longitude/latitude
GeoJSON_dict[0]["coordinates"][0]

5.76034690089422

In [6]:
# pour la suite on travaillera sur un échantillon aléatoire de 500 arbres
GeoJSON_dict=GeoJSON_dict.sample(500)

In [7]:
# Mettez les arbres sur une carte en utilisant Folium.
m = folium.Map(location=[45.16667, 5.71667], zoom_start=12, tiles='Stamen Toner')
feature_group = folium.FeatureGroup("Arbres")
for ligne in range(0,500):
    feature_group.add_child(folium
                            .CircleMarker(location=[
                                float(GeoJSON_dict.iloc[ligne]["coordinates"][1]),
                                float(GeoJSON_dict.iloc[ligne]["coordinates"][0])],
                                         radius=2)) 
m.add_child(feature_group)
m

In [8]:
#Calculer la distance entre deux arbres de deux manières différentes
#avec la fct distance de geopy
a=(GeoJSON_dict.iloc[0]["coordinates"][0],GeoJSON_dict.iloc[0]["coordinates"][1])
b=(GeoJSON_dict.iloc[1]["coordinates"][0],GeoJSON_dict.iloc[1]["coordinates"][1])
print(a,b)
distance.distance(a, b).meters


(5.70725033690709, 45.1619353313426) (5.73868097342763, 45.1647358269934)


3489.5821559374567

In [9]:
# avec la formule d'Haverstine
def dist_haversine(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = deg2rad(lat2-lat1)
    dLon = deg2rad(lon2-lon1)
    a = (np.sin(dLat/2) * np.sin(dLat/2) +
    np.cos(deg2rad(lat1)) * np.cos(deg2rad(lat2)) *
    np.sin(dLon/2) * np.sin(dLon/2))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c # Distance in km
    return d

def deg2rad(deg):
    return deg*(np.pi/180)

print(dist_haversine(a[0], a[1], b[0], b[1]))

3.5086355262768656


In [10]:
# on convertie le dictionnaire en series

# data_lat_long=pd.DataFrame()
# data_lat_long["latitude"]=GeoJSON_dict.apply(lambda x: x["coordinates"][0])
# data_lat_long["longitude"]=GeoJSON_dict.apply(lambda x: x["coordinates"][1])
# data_lat_long

# ou plus simplement
azerty=GeoJSON_dict.apply(pd.Series)
azerty
# on pourrait extraire uniquement les longitudes et latitudes
# azer=azerty["coordinates"].apply(pd.Series)

,type,coordinates
25827,Point,"[5.70725033690709, 45.1619353313426]"
2647,Point,"[5.73868097342763, 45.1647358269934]"
745,Point,"[5.7041019541904, 45.1685922377594]"
21678,Point,"[5.74333560210886, 45.1783402713387]"
11708,Point,"[5.743627491861, 45.1939525095071]"
...,...,...
29169,Point,"[5.73487799959527, 45.1847262374787]"
380,Point,"[5.70631584202629, 45.1761417179879]"
3199,Point,"[5.70163542920322, 45.2044474456057]"
3910,Point,"[5.74031219458772, 45.1914940631663]"


In [11]:
# test de fct lambda pour réaliser des tuples de coordonnées
azerty["coordinates"].apply(lambda x: tuple(x))

25827    (5.70725033690709, 45.1619353313426)
2647     (5.73868097342763, 45.1647358269934)
745       (5.7041019541904, 45.1685922377594)
21678    (5.74333560210886, 45.1783402713387)
11708      (5.743627491861, 45.1939525095071)
                         ...                 
29169    (5.73487799959527, 45.1847262374787)
380      (5.70631584202629, 45.1761417179879)
3199     (5.70163542920322, 45.2044474456057)
3910     (5.74031219458772, 45.1914940631663)
26109    (5.72224336504918, 45.1762194367511)
Name: coordinates, Length: 500, dtype: object

In [12]:
# Quels sont les 10 arbres les plus proches de chez vous ?
# faire un dataframe avec mes coordonnées
# myHome=pd.DataFrame(data=[[5.720382352069388,45.18841421604648]],columns=['latitude','longitude'])
# revient à faire :
# myHome=pd.DataFrame(index=range(1))
# myHome["latitude"]=5.720382352069388
# myHome["longitude"]=45.18841421604648
myhomeTuple=(5.720382352069388,45.18841421604648)

In [13]:
#on calcul les distances entre les arbres et chez moi
azerty["coordinates"].apply(lambda y: distance.distance(y,myhomeTuple).meters).argsort()

25827     28
2647     283
745      225
21678     87
11708    237
        ... 
29169    338
380      296
3199     445
3910     266
26109    436
Name: coordinates, Length: 500, dtype: int64

In [14]:
# Quelle est la distance moyenne entre les arbres ? 2 méthodes

In [15]:
# méthode 1
#%%timeit
azerty_comboBIS=azerty["coordinates"][0:500].apply(lambda x: azerty["coordinates"][0:500]
                            .transpose().apply(lambda y: distance.distance(x, y).meters))
azerty_comboBIS

,25827,2647,745,21678,11708,28379,23780,7794,11275,14545,...,22033,23149,3558,5700,6826,29169,380,3199,3910,26109
25827,0.000000,3489.582156,815.458305,4384.757239,5362.856777,4217.371689,1675.686459,4786.911356,1545.189833,3319.864828,...,2647.240307,3524.656810,3249.194566,3023.197682,2916.086898,3963.272106,1577.051377,4749.906815,4907.916493,2291.834089
2647,3489.582156,0.000000,3847.717073,1592.382224,3282.085844,2553.715210,3849.570394,2724.220397,2016.419831,506.371158,...,2239.099030,140.329301,355.799979,3873.815612,4475.638196,2253.835160,3795.553600,6011.025677,2969.364159,2218.612103
745,815.458305,3847.717073,0.000000,4471.009510,5195.792058,4054.156868,959.839481,4658.651101,2106.940813,3581.177224,...,2492.962933,3908.733564,3557.676731,2428.379837,2181.521342,3844.070530,871.369258,3981.114728,4740.246869,2176.816341
21678,4384.757239,1592.382224,4471.009510,0.000000,1729.580957,1343.069258,4118.380502,1239.237493,3193.242738,1304.923504,...,2119.719440,1706.100790,1499.946854,3534.145219,4298.542550,1172.644923,4101.079181,5443.165936,1494.845979,2344.292549
11708,5362.856777,3282.085844,5195.792058,1729.580957,0.000000,1147.980860,4552.368348,608.368149,4515.056009,2901.390343,...,2725.988217,3406.562928,3116.003518,3430.284091,4237.139764,1407.325515,4573.518221,4787.005745,456.689095,3074.140144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29169,3963.272106,2253.835160,3844.070530,1172.644923,1407.325515,320.752632,3293.043857,824.156586,3138.150612,1778.791001,...,1351.528199,2392.970352,1990.151240,2449.019307,3244.814715,0.000000,3298.586726,4276.219820,960.778049,1685.249175
380,1577.051377,3795.553600,871.369258,4101.079181,4573.518221,3461.650982,99.777042,4080.438037,2424.202474,3435.374847,...,1997.289995,3883.344534,3465.153203,1569.535757,1339.158632,3298.586726,0.000000,3177.913944,4126.221066,1761.370701
3199,4749.906815,6011.025677,3981.114728,5443.165936,4787.005745,4181.228170,3078.311033,4647.482673,5375.972964,5521.784422,...,3776.192154,6138.996109,5657.912073,2163.663862,1854.357397,4276.219820,3177.913944,0.000000,4511.337574,3869.186172
3910,4907.916493,2969.364159,4740.246869,1494.845979,456.689095,691.533690,4107.262756,255.628570,4088.542255,2554.811568,...,2269.437959,3099.968462,2771.341109,3029.055067,3839.817072,960.778049,4126.221066,4511.337574,0.000000,2618.241619


In [16]:
# # méthode 2 : avec itertools.combinations pour éviter les doublons 
azerty_combo=[distance.distance(a,b).meters for a, b 
              in combinations(azerty["coordinates"][0:500].tolist(),2)]

# on met en forme les résultats dans un DataFrame et on met en index quels arbres sont 
# comparés entre eux
df_azerty_combo=pd.DataFrame(azerty_combo)
df_azerty_combo["combo"]=list((i,j) for ((i,_),(j,_)) 
                                         in combinations
                                          (enumerate(azerty["coordinates"][0:500].tolist()), 2))
df_azerty_combo.columns=["Distance","Combo"]
df_azerty_combo.set_index("Combo", inplace=True)
df_azerty_combo

,Distance
Combo,
"(0, 1)",3489.582156
"(0, 2)",815.458305
"(0, 3)",4384.757239
"(0, 4)",5362.856777
"(0, 5)",4217.371689
...,...
"(496, 498)",4126.221066
"(496, 499)",1761.370701
"(497, 498)",4511.337574


In [17]:
#distance moyenne entre les x (taille sample) arbres de la liste (en mètres) ?
df_azerty_combo.mean()

Distance    2460.445141
dtype: float64

In [18]:
#Quel est l’arbre le plus entouré ?

#on crée un indice de proximité en sommant l'inverse de la distance des arbres proches
df_proximity= azerty_comboBIS[azerty_comboBIS!=0]
df_proximity=1/df_proximity
df_proximity

,25827,2647,745,21678,11708,28379,23780,7794,11275,14545,...,22033,23149,3558,5700,6826,29169,380,3199,3910,26109
25827,NaN,0.000287,0.001226,0.000228,0.000186,0.000237,0.000597,0.000209,0.000647,0.000301,...,0.000378,0.000284,0.000308,0.000331,0.000343,0.000252,0.000634,0.000211,0.000204,0.000436
2647,0.000287,NaN,0.000260,0.000628,0.000305,0.000392,0.000260,0.000367,0.000496,0.001975,...,0.000447,0.007126,0.002811,0.000258,0.000223,0.000444,0.000263,0.000166,0.000337,0.000451
745,0.001226,0.000260,NaN,0.000224,0.000192,0.000247,0.001042,0.000215,0.000475,0.000279,...,0.000401,0.000256,0.000281,0.000412,0.000458,0.000260,0.001148,0.000251,0.000211,0.000459
21678,0.000228,0.000628,0.000224,NaN,0.000578,0.000745,0.000243,0.000807,0.000313,0.000766,...,0.000472,0.000586,0.000667,0.000283,0.000233,0.000853,0.000244,0.000184,0.000669,0.000427
11708,0.000186,0.000305,0.000192,0.000578,NaN,0.000871,0.000220,0.001644,0.000221,0.000345,...,0.000367,0.000294,0.000321,0.000292,0.000236,0.000711,0.000219,0.000209,0.002190,0.000325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29169,0.000252,0.000444,0.000260,0.000853,0.000711,0.003118,0.000304,0.001213,0.000319,0.000562,...,0.000740,0.000418,0.000502,0.000408,0.000308,NaN,0.000303,0.000234,0.001041,0.000593
380,0.000634,0.000263,0.001148,0.000244,0.000219,0.000289,0.010022,0.000245,0.000413,0.000291,...,0.000501,0.000258,0.000289,0.000637,0.000747,0.000303,NaN,0.000315,0.000242,0.000568
3199,0.000211,0.000166,0.000251,0.000184,0.000209,0.000239,0.000325,0.000215,0.000186,0.000181,...,0.000265,0.000163,0.000177,0.000462,0.000539,0.000234,0.000315,NaN,0.000222,0.000258
3910,0.000204,0.000337,0.000211,0.000669,0.002190,0.001446,0.000243,0.003912,0.000245,0.000391,...,0.000441,0.000323,0.000361,0.000330,0.000260,0.001041,0.000242,0.000222,NaN,0.000382


In [19]:
#Quel est l’arbre le plus entouré ? 242 avec indice prox = 2.883
#Quel est l’arbre le plus solitaire ? 75 avec indice prox = 0.170
print(df_proximity.sum().argsort())
print(df_proximity.sum().describe())


25827    419
2647     411
745      427
21678    479
11708    468
        ... 
29169    157
380      459
3199     182
3910     460
26109    304
Length: 500, dtype: int64
count    500.000000
mean       0.366116
std        0.105034
min        0.149738
25%        0.297051
50%        0.354456
75%        0.418811
max        0.840670
dtype: float64


In [20]:
# L'arbre le plus entouré est en vert et le plus éloigné est en jaune
i = df_proximity.sum().argsort().iloc[0]
folium.CircleMarker(location=[azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]],
                    radius=4, color= 'yellow').add_to(m)
i = df_proximity.sum().argsort().iloc[499]
folium.CircleMarker(location=[azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]],
                    radius=4, color= 'green').add_to(m)
m

In [21]:
azerty

,type,coordinates
25827,Point,"[5.70725033690709, 45.1619353313426]"
2647,Point,"[5.73868097342763, 45.1647358269934]"
745,Point,"[5.7041019541904, 45.1685922377594]"
21678,Point,"[5.74333560210886, 45.1783402713387]"
11708,Point,"[5.743627491861, 45.1939525095071]"
...,...,...
29169,Point,"[5.73487799959527, 45.1847262374787]"
380,Point,"[5.70631584202629, 45.1761417179879]"
3199,Point,"[5.70163542920322, 45.2044474456057]"
3910,Point,"[5.74031219458772, 45.1914940631663]"


In [22]:
# on ajoute une visualisation en Heatmap
from folium.plugins import HeatMap
# on inverse latitude et longitude pour l'entré en paramètre de HeatMap
list_inverse=[]
for i in range(500):
    list_inverse.append([azerty["coordinates"].iloc[i][1],
                            azerty["coordinates"].iloc[i][0]])
HeatMap(list_inverse).add_to(m)
m